In [ ]:
# ------------------------------------------------------------
# Dashboard Setup (JupyterDash)
# ------------------------------------------------------------
# JupyterDash lets the Dash app run inside a Jupyter notebook
# so you can view the dashboard inline instead of opening a new browser tab.
from jupyter_dash import JupyterDash

# ------------------------------------------------------------
# Dashboard Imports
# ------------------------------------------------------------
# These imports provide the dashboard UI components, charts, and map tools.
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Used to embed the logo image directly into the dashboard
import base64

# This helps JupyterDash work correctly in hosted notebook environments.
JupyterDash.infer_jupyter_proxy_config()

# ------------------------------------------------------------
# General Libraries
# ------------------------------------------------------------
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ------------------------------------------------------------
# CRUD Module Import (Data Access Layer)
# ------------------------------------------------------------
# The dashboard does not talk to MongoDB directly.
# Instead, it uses the AnimalShelter CRUD class as an abstraction layer.
# This keeps database logic separate from the UI code.
from CRUD_Python_Module import AnimalShelter


# ------------------------------------------------------------
# Data Loading / Model Layer
# ------------------------------------------------------------
# MongoDB credentials for the course project.
# (In a real system, these should be stored more securely.)
username = "aacuser"
password = "MyStrongPassword123"

# Create an AnimalShelter object to handle database operations
db = AnimalShelter(username, password)

# Load all records for the initial dashboard view
df = pd.DataFrame.from_records(db.read({}))

# MongoDB includes an _id field (ObjectId type).
# Dash DataTable can have trouble displaying ObjectId values,
# so we remove it to prevent display issues.
df.drop(columns=['_id'], inplace=True, errors='ignore')

# Optional debugging (useful when verifying data shape/columns)
# print("Rows loaded:", len(df))
# print("Columns:", list(df.columns))


# ------------------------------------------------------------
# Dashboard Layout / View Layer
# ------------------------------------------------------------
app = JupyterDash(__name__)

# Load and encode the Grazioso Salvare logo so it can be displayed in Dash.
# Base64 encoding lets us embed the image directly without needing a web URL.
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# ------------------------------------------------------------
# App Layout
# ------------------------------------------------------------
# The layout includes:
# - A title and logo (project requirement)
# - Filter options for rescue categories
# - A data table for browsing records
# - A chart showing breed distribution
# - A map showing the selected animal’s location
app.layout = html.Div([

    # Dashboard title
    html.Center(html.B(html.H1('CS-340 Grazioso Salvare Dashboard'))),

    # Logo + unique identifier (name/project)
    html.Center(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '100px'}
        )
    ),
    html.Center(html.H4('Stevan Ledesma – Project Two')),

    html.Hr(),

    # Rescue category filters (each maps to a specific CRUD query method)
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain / Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster / Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset (Show All)', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ),

    html.Hr(),

    # Interactive table view of the data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    html.Hr(),

    # Chart + Map area (side-by-side)
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])


# ------------------------------------------------------------
# Controller Layer (Callbacks)
# ------------------------------------------------------------
# These callbacks connect user actions (filters/selections) to updates
# in the table, chart, and map.

# Update the table when the user selects a rescue category filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):

    # Choose the database query based on the selected filter
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.read_water_rescue())
    elif filter_type == 'mountain':
        df = pd.DataFrame.from_records(db.read_mountain_rescue())
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(db.read_disaster_tracking())
    else:
        df = pd.DataFrame.from_records(db.read({}))

    # Remove _id again after each query to keep DataTable stable
    df.drop(columns=['_id'], inplace=True, errors='ignore')
    return df.to_dict('records')


# Update the chart based on the records currently visible in the table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):

    # If there is no data (or filters removed everything), show a message
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available to display.")

    # Convert table
in table data to DataFrame for plotting
    dff = pd.DataFrame.from_dict(viewData)

    # Pie chart showing breed distribution
    fig = px.pie(dff, names='breed', title='Breed Distribution of Available Rescue Dogs')
    return dcc.Graph(figure=fig)


# Highlight selected columns in the table (small UI improvement)
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):

    if selected_columns is None:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Update the map based on the selected row in the table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):

    # If no data is available, don’t build the map
    if viewData is None:
        return

    # Default to the first row if the user hasn’t selected anything
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    dff = pd.DataFrame.from_dict(viewData)

    # Build the map and place a marker using the selected row data
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]


# Run the dashboard inline inside the notebook
app.run_server(mode='inline')


Successfully connected to MongoDB
Query successful. Found 10004 documents


Query successful. Found 10004 documents
Query successful. Found 14 documents
Query successful. Found 10004 documents
Query successful. Found 5 documents
Query successful. Found 18 documents
Query successful. Found 5 documents
Query successful. Found 18 documents
Query successful. Found 14 documents
Query successful. Found 10004 documents
Query successful. Found 14 documents
Query successful. Found 10004 documents
